In [2]:
# for sample in samples:
#     sample["conditions"] = sample["conditions"][0]
#     sample["procedures"] = sample["procedures"][0]
#     sample["drugs"] = sample["drugs"][0]

In [9]:
TASK = "mortality"
DATASET = "mimic3"


In [10]:
from pyhealth.datasets import SampleEHRDataset
import json

with open(f"/shared/eng/pj20/kelpie_exp_data/ehr_data/{DATASET}_{TASK}_samples_train.json", "r") as f:
    samples_train = json.load(f)
with open(f"/shared/eng/pj20/kelpie_exp_data/ehr_data/{DATASET}_{TASK}_samples_test.json", "r") as f:
    samples_test = json.load(f)


dataset_train = SampleEHRDataset(samples_train, dataset_name=DATASET, task_name=TASK)
dataset_test = SampleEHRDataset(samples_test, dataset_name=DATASET, task_name=TASK)


In [11]:
len(dataset_test)

996

In [12]:
from pyhealth.datasets import split_by_patient, get_dataloader

# train_dataset, val_dataset, test_dataset = split_by_patient(
#     dataset, [0.8, 0.1, 0.1], seed=528
# )
train_dataloader = get_dataloader(dataset_train, batch_size=32, shuffle=True)
test_dataloader = get_dataloader(dataset_test, batch_size=32, shuffle=False)

In [18]:
from pyhealth.trainer import Trainer
from pyhealth.models import Deepr, AdaCare, StageNet, GRASP, Transformer, RETAIN, RNN
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

model = GRASP(
    dataset=dataset_train,
    feature_keys=["conditions", "procedures", "drugs"],
    label_key="label",
    mode="binary",
    use_embedding=[True, True, True],
    embedding_dim=128,
)


In [19]:
trainer = Trainer(model=model, metrics=['accuracy', 'f1', 'pr_auc', 'sensitivity', 'specificity'])
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=test_dataloader,
    epochs=15,
    optimizer_params = {"lr": 1e-3},
    monitor="f1",
)

# STEP 5: evaluate
print(trainer.evaluate(test_dataloader))

GRASP(
  (embeddings): ModuleDict(
    (conditions): Embedding(263, 128, padding_idx=0)
    (procedures): Embedding(192, 128, padding_idx=0)
    (drugs): Embedding(194, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (grasp): ModuleDict(
    (conditions): GRASPLayer(
      (backbone): ConCareLayer(
        (PositionalEncoding): PositionalEncoding(
          (dropout): Dropout(p=0, inplace=False)
        )
        (GRUs): ModuleList(
          (0-127): 128 x GRU(1, 128, batch_first=True)
        )
        (LastStepAttentions): ModuleList(
          (0-127): 128 x SingleAttention(
            (tanh): Tanh()
            (softmax): Softmax(dim=1)
            (sigmoid): Sigmoid()
            (relu): ReLU()
          )
        )
        (FinalAttentionQKV): FinalAttentionQKV(
          (W_q): Linear(in_features=128, out_features=128, bias=True)
          (W_k): Linear(in_features=128, out_features=128, bias=True)
          (W_v): Linear(in_features=128, out_features=128, bias=True)

Epoch 0 / 15: 100%|██████████| 242/242 [04:14<00:00,  1.05s/it]

--- Train epoch-0, step-242 ---
loss: 0.2626



Evaluation: 100%|██████████| 32/32 [00:11<00:00,  2.69it/s]

--- Eval epoch-0, step-242 ---
accuracy: 0.9458
f1: 0.4861
pr_auc: 0.0718
sensitivity: 0.0000
specificity: 1.0000
loss: 0.2068
New best f1 score (0.4861) at epoch-0, step-242


...

--- Train epoch-14, step-3630 ---
loss: 0.2121



Evaluation: 100%|██████████| 32/32 [00:15<00:00,  2.13it/s]

--- Eval epoch-14, step-3630 ---
accuracy: 0.9448
f1: 0.4858
pr_auc: 0.1082
sensitivity: 0.0000
specificity: 0.9989
loss: 0.2135
Loaded best model



/home/pj20/PyHealth/pyhealth/trainer.py:337: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(ckpt_path, map_location=self.device)
Evaluation: 100%|███

{'accuracy': 0.9447791164658634, 'f1': 0.5033319733442132, 'pr_auc': 0.09635838918523011, 'sensitivity': 0.018518518518518517, 'specificity': 0.9978768577494692, 'loss': 0.2662505491171032}
